# Updating a Historical Marquee Portfolio

If you already have a portfolio in Marquee, the GS Quant SDK provides a simple and intuitive workflow to update positions and rerun reports.

You will need to be an admin on the portfolio you would like to update. If you are not an admin, please ask a portfolio admin to [edit the portfolio's entitlements](../examples/marquee/01_edit_portfolio_entitlements.ipynb) to include you.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt

from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.session import GsSession, Environment
from gs_quant.markets.position_set import PositionSet

client = None
secret = None
scopes = None

## External users must fill in their client ID and secret below and comment out the line below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(
    Environment.PROD,
    client_id=client,
    client_secret=secret,
)

print('GS Session initialized.')

## Step 2: Define Your Portfolio ID and the Positions You Would Like to Upload

Portfolio positions in Marquee are stored on a holding basis, when means you only upload positions for days where you are rebalancing your portfolio. 

Positions can be uploaded using the following identifiers:

| Identifier                        | Descriptions                                                              |
|-----------------------------------|---------------------------------------------------------------------------|
| BLOOMBERG ID = "BBID"             | Bloomberg identifier and exchange code (GS UN)                            |
| SEDOL = "SEDOL"                   | LSE Stock Exchange Daily Official List code (2407966)                     |
| CUSIP = "CUSIP"                   | Committee on Uniform Security Identification Procedures code (38141G104)  |
| ISIN = "ISIN"                     | International Securities Identification Number (US38141G1040)             |
| GSID = "GSID"                     | Goldman Sachs Identifier                                                  |
| REUTERS ID = "RIC"                | Thompson Reuters Instrument Code (RIC), (GS.N)                            |
| TICKER = "TICKER"                 | Exchange ticker (GS)                                                      |

 Depending on your quantity type, you can upload by shares, weights, or exposures.


In [ ]:
portfolio_id = 'PORTFOLIO ID'

### Uploading Positions Using Shares
Portfolio quantity type uses share counts.

In [ ]:
portfolio_position_sets = [
    PositionSet.from_dicts(
        date=dt.date(day=3, month=5, year=2021),
        positions=[{'identifier': 'AAPL UW', 'quantity': 25}, {'identifier': 'GS UN', 'quantity': 50}],
    ),
    PositionSet.from_dicts(
        date=dt.date(day=1, month=7, year=2021),
        positions=[{'identifier': 'AAPL UW', 'quantity': 25}, {'identifier': 'GS UN', 'quantity': 50}],
    ),
]

### Uploading Positions Using Exposures
Portfolio quantity type uses notional values.

In [ ]:
portfolio_position_sets = [
    PositionSet.from_dicts(
        date=dt.date(day=3, month=5, year=2021),
        positions=[{'identifier': 'AAPL UW', 'notional': 2500}, {'identifier': 'GS UN', 'notional': 5000}],
    ),
    PositionSet.from_dicts(
        date=dt.date(day=1, month=7, year=2021),
        positions=[{'identifier': 'AAPL UW', 'notional': 2500}, {'identifier': 'GS UN', 'notional': 500}],
    ),
]

### Uploading Positions Using Weights
Portfolio quantity type uses weight values.

In [ ]:
portfolio_position_sets = [
    PositionSet.from_dicts(
        date=dt.date(day=3, month=5, year=2021),
        reference_notional=10000,
        positions=[{'identifier': 'AAPL UW', 'weight': 0.5}, {'identifier': 'GS UN', 'weight': 0.5}],
    ),
    PositionSet.from_dicts(
        date=dt.date(day=1, month=7, year=2021),
        reference_notional=10000,
        positions=[{'identifier': 'AAPL UW', 'weight': 0.5}, {'identifier': 'GS UN', 'weight': 0.5}],
    ),
]

### Resolve Positions

In [ ]:
for pos_set in portfolio_position_sets:
    pos_set.resolve()
    if len(pos_set.unresolved_positions) > 0:
        print(
            f'The following positions on {pos_set.date} could not be resolved: {[p.identifier for p in pos_set.unresolved_positions]}'
        )
    else:
        print(f"All positions on {pos_set.date} have been resolved and saved.")

## Price Positions (Weights & Exposures)
When uploading positions by weights or exposures they must priced and converted to share quantities before being uploaded to the portfolio.
Select the currency you want to price your positions with. If you do not specify, default is USD.

In [ ]:
currency = 'USD'

for pos_set in portfolio_position_sets:
    pos_set.price(currency)

## Step 3: Post Positions to the Marquee Portfolio

In [ ]:
pm = PortfolioManager(portfolio_id)
pm.update_positions(portfolio_position_sets)

#### Have your positions as a dataframe?

If you have a day's positions in a dataframe with columns `identifer` (string values), `quantity` (float values),
and optionally `tags` (dictionary values), you can turn them into a `PositionSet` object by using the
`PositionSet.from_frame()` function:

`position_set = PositionSet.from_frame(positions_df, datetime_date)`

You can also substitute the `quantity` column with either `notional` (float values) or `weight` (float values) columns.

In [ ]:
# positions_df = DATAFRAME WITH POSITIONS
# datetime_date = POSITIONS' DATE

portfolio_position_sets = []
portfolio_position_sets.append(PositionSet.from_frame(positions_df, datetime_date))

for pos_set in portfolio_position_sets:
    pos_set.resolve()
    if len(pos_set.unresolved_positions) > 0:
        print(
            f'The following positions on {pos_set.date} could not be resolved: {[p.identifier for p in pos_set.unresolved_positions]}'
        )
    else:
        print(f"All positions on {pos_set.date} have been resolved and saved.")

## Step 4: Reschedule All Portfolio Reports

Now that the portfolio has new positions, it's time to rerun all reports associated with the portfolio so your performance, risk, and other analytics reflect these new positions.

In [ ]:
pm.schedule_reports()

print('All portfolio reports scheduled.')

*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*